### Using different methods from week 2

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob

In [2]:

def extract_keypoints_SIFT(img1, img2):
    """
    use SIFT to detect keypoint features and match between images.
    Includes a Lowes ratio test to improve performance.

    params:
        - img1, img2: images to match.
        - K: projection matrices for images
        - baseline: base translation between cameras.

    returns:
        - initial landmark points.
        - corresponding 2D points in img1.
    """

    # 1 SIFT
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # 2 KNN match with two (best and second best)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # 3 Ratio test!
    kp1_indexes = []
    kp2_indexes = []
    good_matches= []
    for m,n in matches:
        if m.distance < 0.70*n.distance:
            good_matches.append([m])
            kp1_indexes.append(m.queryIdx)
            kp2_indexes.append(m.trainIdx)

    ## 4 extract the keypoints of the "good" matches
    kp1 = np.asarray(kp1)
    kp2 = np.asarray(kp2)
    match_points1 = [p.pt for p in kp1[kp1_indexes]]
    match_points2 = [p.pt for p in kp2[kp2_indexes]]

    p1 = np.array(match_points1).astype(np.float32)
    p2 = np.array(match_points2).astype(np.float32)

    return p1, p2

#### 1 - spare optical matching

In [4]:
images = glob.glob("video/sample/left/*.png")
no_of_frames = len(images)-1


image_buffer = [] # Sliding image buffer
buffer_len = 10
q = 0 # Queue index
frame_difference = 15
max_frame_dist = 70

# Get frame
compare_frame = cv2.imread(images[1])
gray_compare = cv2.cvtColor(compare_frame, cv2.COLOR_BGR2GRAY)

buffer = []

for k in range(1, no_of_frames, 1):
    # Get frame
    frame = cv2.imread(images[k])
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    buffer.append(gray)

    # Extract and track features
    feat1 = cv2.goodFeaturesToTrack(gray, maxCorners=10000, qualityLevel=0.001, minDistance=10)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(gray, gray_compare, feat1, None)
    # Remove untracked features    
    good_indexes = np.where(np.any(status==1, axis=1))[0]
    feat1 = feat1[good_indexes]
    feat2 = feat2[good_indexes]
    # Draw the best features
    for i in range(len(feat1)):
        f10=int(feat1[i][0][0])
        f11=int(feat1[i][0][1])
        f20=int(feat2[i][0][0])
        f21=int(feat2[i][0][1])
        x = (f20-f10)
        y = (f21-f11)
        distance = np.sqrt(x**2 + y**2)
        if max_frame_dist > distance  and distance > frame_difference:
            if y==0:
                continue
            angle = np.arctan(x/y)
            if angle<0:
                cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)
                cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)

    if k%10==0:
        gray_compare = buffer[-9].copy()

    cv2.imshow('image', frame)
    cv2.waitKey

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()